# The Battle of Neighborhoods - Code
### Applied Data Science Capstone by IBM/Coursera

#### Importing needed libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Using Nominatim to obtain coordinates of cities

In [4]:
address = 'Los Angeles'

geolocator = Nominatim(user_agent="foursquare_agent")
la_location = geolocator.geocode(address)
la_latitude = la_location.latitude
la_longitude = la_location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(la_latitude, la_longitude))

The geograpical coordinate of Los Angeles are 34.0536909, -118.2427666.


In [5]:
address = 'San Francisco'

geolocator = Nominatim(user_agent="foursquare_agent")
sf_location = geolocator.geocode(address)
sf_latitude = sf_location.latitude
sf_longitude = sf_location.longitude
print('The geograpical coordinate of San Francisco are {}, {}.'.format(sf_latitude, sf_longitude))

The geograpical coordinate of San Francisco are 37.7790262, -122.4199061.


In [7]:
address = 'Portland'

geolocator = Nominatim(user_agent="foursquare_agent")
por_location = geolocator.geocode(address)
por_latitude = por_location.latitude
por_longitude = por_location.longitude
print('The geograpical coordinate of Portland are {}, {}.'.format(por_latitude, por_longitude))

The geograpical coordinate of Portland are 45.5202471, -122.6741949.


In [8]:
address = 'Seattle'

geolocator = Nominatim(user_agent="foursquare_agent")
sea_location = geolocator.geocode(address)
sea_latitude = sea_location.latitude
sea_longitude = sea_location.longitude
print('The geograpical coordinate of Seattle are {}, {}.'.format(sea_latitude, sea_longitude))

The geograpical coordinate of Seattle are 47.6038321, -122.3300624.


#### Define Foursquare Credentials, Version, Limit and Radius

In [11]:
CLIENT_ID = 'F0QECAXOFELPTLKYCS0AU1NWT2DVQFRSUOG04VX2ZBE3ZL2L' # your Foursquare ID
CLIENT_SECRET = 'EGGXFSJKZU0KMPVH504GQM4KRTAXPMKC4ZJZRT2YATJKJ0IU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 250 # Get top 250 venues
radius = 100000 # Define a radius of 100 kilometers/62 miles


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: F0QECAXOFELPTLKYCS0AU1NWT2DVQFRSUOG04VX2ZBE3ZL2L
CLIENT_SECRET:EGGXFSJKZU0KMPVH504GQM4KRTAXPMKC4ZJZRT2YATJKJ0IU


### Get the top 100 venues that are in each city within a radius of 100 km/62 miles

In [12]:
la_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    la_latitude, 
    la_longitude, 
    radius, 
    LIMIT)

la_results = requests.get(la_url).json()

In [21]:
sf_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    sf_latitude, 
    sf_longitude, 
    radius, 
    LIMIT)

sf_results = requests.get(sf_url).json()

In [23]:
por_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    por_latitude, 
    por_longitude, 
    radius, 
    LIMIT)

por_results = requests.get(por_url).json()

In [24]:
sea_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    sea_latitude, 
    sea_longitude, 
    radius, 
    LIMIT)

sea_results = requests.get(sea_url).json()

In [13]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
la_venues = la_results['response']['groups'][0]['items']
    
la_nearby_venues = json_normalize(la_venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.location.distance']
la_nearby_venues = la_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
la_nearby_venues['venue.categories'] = la_nearby_venues.apply(get_category_type, axis=1)

# clean columns
la_nearby_venues.columns = [col.split(".")[-1] for col in la_nearby_venues.columns]

la_nearby_venues.head(15)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,distance
0,Walt Disney Concert Hall,Concert Hall,34.055511,-118.249284,634
1,The Broad,Art Museum,34.054474,-118.250051,677
2,The Last Bookstore,Bookstore,34.047620,-118.249852,940
3,Hauser & Wirth,Art Gallery,34.046095,-118.234801,1120
4,Salt & Straw,Ice Cream Shop,34.046065,-118.235473,1083
5,Mr. Speedy Plumbing & Rooter Inc.,Construction & Landscaping,34.042538,-118.233864,1488
6,Los Angeles Music Center,Music Venue,34.057133,-118.248354,642
7,KazuNori: The Original Hand Roll Bar,Sushi Restaurant,34.047716,-118.247452,793
8,Vista Hermosa Park,Park,34.061601,-118.256857,1569
9,Guisados,Taco Place,34.070262,-118.250437,1975


In [25]:
sf_venues = sf_results['response']['groups'][0]['items']
    
sf_nearby_venues = json_normalize(sf_venues) # flatten JSON

# filter columns
sf_nearby_venues = sf_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
sf_nearby_venues['venue.categories'] = sf_nearby_venues.apply(get_category_type, axis=1)

# clean columns
sf_nearby_venues.columns = [col.split(".")[-1] for col in sf_nearby_venues.columns]

sf_nearby_venues.head(15)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,distance
0,Asian Art Museum,Art Museum,37.780178,-122.416505,325
1,Salt & Straw,Ice Cream Shop,37.776532,-122.426051,607
2,Birba,Wine Bar,37.777750,-122.424159,400
3,Vive La Tarte,Café,37.777012,-122.410899,823
4,The Olympic Club,Gym / Fitness Center,37.788181,-122.411067,1281
5,Ritual Coffee Roasters,Coffee Shop,37.776476,-122.424281,478
6,Collis P. Huntington Park,Park,37.792162,-122.412154,1613
7,Mr. S Leather & Mr. S Locker Room,Clothing Store,37.774117,-122.408853,1115
8,The Secret Alley,Performing Arts Venue,37.763764,-122.419352,1699
9,Good Vibrations,Adult Boutique,37.763195,-122.421735,1769


In [26]:
por_venues = por_results['response']['groups'][0]['items']
    
por_nearby_venues = json_normalize(por_venues) # flatten JSON

# filter columns
por_nearby_venues = por_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
por_nearby_venues['venue.categories'] = por_nearby_venues.apply(get_category_type, axis=1)

# clean columns
por_nearby_venues.columns = [col.split(".")[-1] for col in por_nearby_venues.columns]

por_nearby_venues.head(15)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,distance
0,Powell's City of Books,Bookstore,45.523180,-122.681375,648
1,Kimpton Hotel Monaco Portland,Hotel,45.519876,-122.677367,250
2,Portland Farmer's Market at PSU,Farmers Market,45.512236,-122.684965,1225
3,Guardian Games,Toy / Game Store,45.515521,-122.662038,1084
4,Tasty ’N Alder,American Restaurant,45.521304,-122.683470,732
5,Nong’s Khao Man Gai,Thai Restaurant,45.522323,-122.659365,1179
6,Canard,French Restaurant,45.522855,-122.657917,1302
7,Pine Street Market,Food Court,45.521421,-122.672494,186
8,Multnomah Whisk{e}y Library,Cocktail Bar,45.521030,-122.683398,723
9,Mediterranean Exploration Company,Mediterranean Restaurant,45.525492,-122.684429,988


In [27]:
sea_venues = sea_results['response']['groups'][0]['items']
    
sea_nearby_venues = json_normalize(sea_venues) # flatten JSON

# filter columns
sea_nearby_venues = sea_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
sea_nearby_venues['venue.categories'] = sea_nearby_venues.apply(get_category_type, axis=1)

# clean columns
sea_nearby_venues.columns = [col.split(".")[-1] for col in sea_nearby_venues.columns]

sea_nearby_venues.head(15)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng,distance
0,Il Corvo,Italian Restaurant,47.602522,-122.331952,203
1,Ellenos Real Greek Yogurt,Frozen Yogurt Shop,47.608848,-122.340476,960
2,Elliott Bay Book Company,Bookstore,47.614689,-122.319885,1429
3,Biscuit B*tch,Breakfast Spot,47.603237,-122.332010,160
4,Pike Place Market,Market,47.609467,-122.341465,1061
5,Seattle Bouldering Project,Climbing Gym,47.593505,-122.310925,1839
6,Pike Place Fish Market,Fish Market,47.608813,-122.340371,951
7,Beecher's Handmade Cheese,Cheese Shop,47.609570,-122.341851,1091
8,REI,Sporting Goods Shop,47.620344,-122.329754,1838
9,CenturyLink Field,Stadium,47.594789,-122.331904,1016


#### Group  venue categories at each city

In [30]:
la_nearby_venues['categories'].value_counts()

Trail                           5
Ice Cream Shop                  5
Park                            4
Hotel                           4
Sandwich Place                  4
American Restaurant             3
Deli / Bodega                   3
Theme Park Ride / Attraction    3
Theme Park                      3
Garden                          3
Farmers Market                  3
Wine Shop                       3
Art Museum                      3
Museum                          3
Bookstore                       2
Art Gallery                     2
Mediterranean Restaurant        2
Music Venue                     2
Bakery                          2
Scenic Lookout                  2
Coffee Shop                     2
Grocery Store                   2
Yoga Studio                     2
Italian Restaurant              2
Sculpture Garden                1
Science Museum                  1
Breakfast Spot                  1
Concert Hall                    1
Japanese Restaurant             1
Taco Place    

In [31]:
sf_nearby_venues['categories'].value_counts()

Park                      15
Bakery                     5
Yoga Studio                4
Coffee Shop                3
Wine Shop                  3
Ice Cream Shop             3
Gym / Fitness Center       3
Garden                     3
Bookstore                  3
Science Museum             2
Trail                      2
Brewery                    2
Art Museum                 2
Scenic Lookout             2
Grocery Store              2
Marijuana Dispensary       2
Sushi Restaurant           2
Climbing Gym               2
French Restaurant          2
Market                     2
Gym                        2
Restaurant                 1
Filipino Restaurant        1
Outdoor Sculpture          1
Spiritual Center           1
Pizza Place                1
Furniture / Home Store     1
Baseball Stadium           1
Historic Site              1
Salon / Barbershop         1
Beach                      1
Theater                    1
Dance Studio               1
Art Gallery                1
Bar           

In [32]:
por_nearby_venues['categories'].value_counts()

Coffee Shop                  8
Park                         8
Brewery                      6
Grocery Store                4
Hotel                        4
Bakery                       3
Sandwich Place               3
Trail                        3
Indie Movie Theater          2
Pet Store                    2
Toy / Game Store             2
Garden                       2
Thai Restaurant              2
Pizza Place                  2
Cocktail Bar                 2
Ice Cream Shop               2
Mexican Restaurant           2
French Restaurant            2
Middle Eastern Restaurant    1
Restaurant                   1
Russian Restaurant           1
German Restaurant            1
American Restaurant          1
Zoo                          1
Korean Restaurant            1
Pub                          1
Beer Bar                     1
Breakfast Spot               1
Tram Station                 1
Latin American Restaurant    1
Taco Place                   1
Italian Restaurant           1
Beer Sto

In [29]:
sea_nearby_venues['categories'].value_counts()

Park                     9
Coffee Shop              6
Brewery                  5
Bakery                   4
Bookstore                3
Beach                    3
Italian Restaurant       3
Cocktail Bar             3
Taco Place               2
Wine Bar                 2
Ice Cream Shop           2
Poke Place               2
Beer Store               2
Scenic Lookout           2
Sandwich Place           2
French Restaurant        2
Farmers Market           2
Playground               2
Yoga Studio              2
Museum                   2
Baseball Stadium         1
Sculpture Garden         1
Malay Restaurant         1
Restaurant               1
Gym / Fitness Center     1
Cheese Shop              1
Breakfast Spot           1
Art Gallery              1
Beer Bar                 1
Concert Hall             1
Record Shop              1
Bar                      1
Plaza                    1
Japanese Restaurant      1
Donut Shop               1
Diner                    1
Video Store              1
T

#### Narrow venue categories into 6 major categories based on grouping results above

In [34]:
Shopping = ['Arts & Crafts Store','Music Venue','Antique Shop','Boutique','Cosmetics Shop', "Women's Store",'Souvenir Shop','Gift Shop','Toy / Game Store','Shopping Mall','Sporting Goods Shop','Supermarket','Liquor Store','Clothing Store','Beer Store']
Food = ['Sandwich Place','Bakery','Burrito Place','Steakhouse','Seafood Restaurant','Japanese Restaurant','Thai Restaurant', 'Restaurant','Hot Dog Joint','Indian Restaurant','Fried Chicken Joint','Dessert Shop','Pizza Place','Food Truck','Coffee Shop','Hookah Bar','Malay Restaurant','Korean Restaurant','Vietnamese Restaurant','Filipino Restaurant','Arepa Restaurant','American Restaurant','Cajun / Creole Restaurant','Asian Restaurant','African Restaurant','Middle Eastern Restaurant','Russian Restaurant','German Restaurant','Mexican Restaurant','Brazilian Restaurant','Greek Restaurant','Café','French Restaurant','Italian Restaurant','Bagel Shop','Bed & Breakfast','Breakfast Spot','Chinese Restaurant', 'Fast Food Restaurant', 'Donut Shop','Restaurant','Bar', 'Wine Bar','Cocktail Bar','Farmers Market','Diner','Fish Market','Bubble Tea Shop','BBQ Joint','Sushi Restaurant','Frozen Yogurt Shop','Ice Cream Shop','Deli / Bodega','Mediterranean Restaurant','Food Truck','Smoothie Shop','Chocolate Shop']
Entertainment = ['Surf Spot','Theater','Movie Theater','Auditorium','Comedy Club','Strip Club','Tour Provider','Bookstore','Bike Rental / Bike Share','Pool','Tourist Information Center','Golf Course','Spa','Health & Beauty Service','General Entertainment','Gym / Fitness Center','Concert Hall','Zoo','Trail','Gym','Mini Golf','Theme Park']
Museum = ['Public Art','Museum','Art Museum','Performing Arts Venue','Art Gallery','Science Museum']
Senic = ['Park','Sculpture Garden','Garden Center','Animal Shelter','Beer Garden','Scenic Lookout','Beach','River', 'Bridge','Lake','Monument / Landmark','Hill','Botanical Garden','Fountain','Waterfront','Construction & Landscaping','Observatory','Harbor / Marina','Historic Site','Outdoor Sculpture ']
Hotel = ['Plaza','Hotel']


In [41]:
all_cat = Shopping + Food + Entertainment + Museum + Senic + Hotel
all_cat_list = [Shopping,Food,Entertainment,Museum,Senic,Hotel]
all_cat_names = ['Shopping', 'Food','Entertainment', 'Museum', 'Senic', 'Hotel']

all_nearyby_venues = [la_nearby_venues,sf_nearby_venues,por_nearby_venues,sea_nearby_venues]
cats_count = [0,0,0,0]
eachcat_count = [0,0,0,0]

for i, f in enumerate(all_nearyby_venues):
    for j in range(len(f)):
        if f.iloc[j]['categories'] in all_cat:
            cats_count[i]+=1

cities = ['Los Angeles','San Francisco', 'Portland', 'Seattle']
df_count = pd.DataFrame(cities, columns = ['City'])            
df_count['Total Count of Categories'] = cats_count

for c, cat in enumerate(all_cat_list):
    for i, f in enumerate(all_nearyby_venues):
        for j in range(len(f)):
            if f.iloc[j]['categories'] in cat:
                eachcat_count[i]+=1
    df_count[all_cat_names[c]] = eachcat_count
df_count

,City,Total Count of Categories,Shopping,Food,Entertainment,Museum,Senic,Hotel
0,Los Angeles,77,4,38,53,62,73,77
1,San Francisco,67,1,28,40,46,66,67
2,Portland,69,5,51,56,56,65,69
3,Seattle,76,4,44,52,56,74,76


#### Visualizing places in 6 different colors representing 6 major categories in each city

Colors of 6 major categories:
- Shopping: Red
- Food: Green
- Entertainment: Purple
- Museum: Lightgray
- Senic: Blue
- Hotel: Yellow

In [46]:
map_la = folium.Map(location=[la_latitude,la_longitude], zoom_start=10)

color = ''
for lat, lng, label in zip(la_nearby_venues['lat'],la_nearby_venues['lng'],la_nearby_venues['categories']):
    if label in Shopping:
        color = 'red'
    elif label in Food:
        color = 'green'
    elif label in Entertainment:
        color = 'purple'
    elif label in Museum:
        color = 'lightgray'
    elif label in Senic:
        color = 'blue'
    elif label in Hotel:
        color = 'yellow'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_la)

map_la

In [49]:
map_sf = folium.Map(location=[sf_latitude,sf_longitude], zoom_start=12)

color = ''
for lat, lng, label in zip(sf_nearby_venues['lat'],sf_nearby_venues['lng'],sf_nearby_venues['categories']):
    if label in Shopping:
        color = 'red'
    elif label in Food:
        color = 'green'
    elif label in Entertainment:
        color = 'purple'
    elif label in Museum:
        color = 'lightgray'
    elif label in Senic:
        color = 'blue'
    elif label in Hotel:
        color = 'yellow'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_sf)

map_sf

In [50]:
map_por = folium.Map(location=[por_latitude,por_longitude], zoom_start=12)

color = ''
for lat, lng, label in zip(por_nearby_venues['lat'],por_nearby_venues['lng'],por_nearby_venues['categories']):
    if label in Shopping:
        color = 'red'
    elif label in Food:
        color = 'green'
    elif label in Entertainment:
        color = 'purple'
    elif label in Museum:
        color = 'lightgray'
    elif label in Senic:
        color = 'blue'
    elif label in Hotel:
        color = 'yellow'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_por)

map_por

In [51]:
map_sea = folium.Map(location=[sea_latitude,sea_longitude], zoom_start=12)

color = ''
for lat, lng, label in zip(sea_nearby_venues['lat'],sea_nearby_venues['lng'],sea_nearby_venues['categories']):
    if label in Shopping:
        color = 'red'
    elif label in Food:
        color = 'green'
    elif label in Entertainment:
        color = 'purple'
    elif label in Museum:
        color = 'lightgray'
    elif label in Senic:
        color = 'blue'
    elif label in Hotel:
        color = 'yellow'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=color,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_sea)

map_sea